<a href="https://colab.research.google.com/github/rg81073/Exploratory_Data_Analysis_Machine_Learning/blob/main/Fake_News_Prediction_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## About the Dataset:
 1. id: unique id for a news article
 2. title: the title of a news article
 3. author: authoe of the news article
 4. text: the text of the article, could be incomplete
 5. label: a label that marks whether the news article is real or fake


      1:-> Fake news
      0:-> Real news



Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Printing the Stopwords present in English

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Prprocessing

Loading Dataset in to the Pandas Dataframe

In [ ]:
news_dataset = pd.read_csv('/content/train.csv')

In [ ]:
news_dataset.shape

(20800, 5)

In [ ]:
# Printing First Five rows of the DataFrame

news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# Counting the number of misssing values in the DataSet

news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
# Replacing the NULL values with Empty String

news_dataset = news_dataset.fillna('')

In [ ]:
# Merging the 'author' and 'title' columns

news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [ ]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [ ]:
# Separating The DataSet & label

X = news_dataset.drop(columns='label',axis=1)
Y = news_dataset['label']

In [ ]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Doing Stemming Procedure:

Stemming is a text preprocessing technique in natural language processing (NLP) that reduces words to their root or base form, called the "stem." It aims to normalize words by removing prefixes or suffixes and reducing related words to their common base. The resulting stemmed form may not always be a valid word, but it helps in grouping together words that have the same semantic meaning.

The purpose of stemming is to simplify the analysis of text data by reducing inflected words to their base form. This can be beneficial in various NLP tasks such as information retrieval, text classification, and sentiment analysis. By reducing words to their common stem, the number of unique word forms is reduced, which can improve computational efficiency and increase the accuracy of certain text analysis tasks.

There are different algorithms available for stemming, with the most common one being the Porter stemming algorithm. Other popular stemming algorithms include the Snowball stemmer (also known as the Porter2 stemmer) and the Lancaster stemmer. These algorithms follow specific rules and heuristics to strip affixes from words and produce the stem.

It's important to note that stemming is a simple rule-based process and may not always produce accurate results. Since stemming operates purely on word structure without considering the context, it can sometimes result in incorrect stemming or produce stems that are not actual words. However, despite its limitations, stemming remains a useful technique for certain NLP applications, especially when the focus is on word-level patterns and not necessarily on semantic meaning. Example 1:
Input: "running"
Stemmed output: "run"

Example 2:
Input: "jumps"
Stemmed output: "jump"

Example 3:
Input: "happiness"
Stemmed output: "happi"

Example 4:
Input: "playing"
Stemmed output: "play"

Example 5:
Input: "dogs"
Stemmed output: "dog"

In [ ]:
port_stem = PorterStemmer()

In [ ]:
# The line of code i have written is an example of list comprehension in Python.
# It applies stemming to a list of words in stemmed_content and filters out any
# words that are present in the English stopwords list.


def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content



1. stemmed_content: It is assumed that stemmed_content is a list of words that have undergone stemming. For example, it could be a list of stemmed words obtained from text preprocessing.

2. stopwords.words('english'): This part retrieves a list of stopwords in English from the NLTK library. Stopwords are commonly used words (e.g., "a", "an", "the") that are often removed during text analysis because they usually do not carry significant meaning.

3. word in stemmed_content: This is a loop that iterates through each word in the stemmed_content list.

4. if not word in stopwords.words('english'): This condition checks if the current word is not present in the English stopwords list.

5. port_stem.stem(word): For each word that passes the condition, the Porter stemming algorithm (assuming port_stem is an instance of PorterStemmer from the NLTK library) is applied to the word. The stem() function reduces words to their base or root form, for example, "running" becomes "run" and "jumps" becomes "jump".

Finally, the list comprehension encapsulates these operations and returns a new list containing the stemmed words that are not present in the stopwords list.

Applying Performed Stemming Operation on 'content' which is a 'list'

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
# Separating the Data and Label

X= news_dataset['content'].values

Y= news_dataset['label'].values

In [ ]:
print(X)


['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [ ]:
print(Y)

[1 0 1 ... 0 1 1]


In [ ]:
Y.shape

(20800,)

Converting the 'Textual' data to 'Numerical' Data using 'TfidfVectorizer':

TfidfVectorizer is a feature extraction technique commonly used in natural language processing (NLP) tasks, such as text classification, information retrieval, and document similarity analysis. **"Tfidf" stands for "Term Frequency-Inverse Document Frequency."**

TfidfVectorizer converts a collection of raw text documents into a matrix of numerical features. It computes the TF-IDF value for each term (word) in the documents, which represents the importance of the term in the context of the entire corpus. The TF-IDF value is a product of two components:

Term Frequency (TF): It measures how frequently a term appears in a document. It is the ratio of the number of times a term occurs in a document to the total number of terms in the document. TF assigns higher weights to terms that appear more often within a document.

Inverse Document Frequency (IDF): It measures the rarity of a term across the entire corpus. It is the logarithmically scaled inverse fraction of the number of documents that contain the term. IDF assigns higher weights to terms that are less common across documents.

TfidfVectorizer performs the following steps:

Tokenization: It breaks the text into individual words or tokens.

Text Preprocessing: It applies optional text preprocessing steps such as lowercasing, removing punctuation, and filtering out stopwords.

Building Vocabulary: It constructs a vocabulary of unique terms (words) from the text corpus.

Encoding: It converts each document in the corpus into a numerical feature vector representation based on the TF-IDF values of the terms.

The resulting feature matrix can be used as input for machine learning algorithms. It represents the documents in a high-dimensional space, where each dimension corresponds to a unique term, and the value in each dimension represents the TF-IDF score of that term in the document.

TfidfVectorizer is available in popular Python libraries like scikit-learn. It provides various parameters to control the feature extraction process, such as specifying n-grams, controlling the IDF weighting scheme, and setting vocabulary size limits.

Overall, TfidfVectorizer is a powerful tool for converting raw text data into a numerical representation that captures the importance of terms in a document within a corpus. It enables machine learning algorithms to work effectively with text data.

In [ ]:
# Converting the 'Textual' data to 'Numerical' Data using 'TfidfVectorizer':

vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)


In [ ]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

**Splitting 'Train' and 'Test' Data: -**

The parameters "stratify" and "random_state" are often used in the train-test split function to control the randomness and ensure representative data distribution between the training and test sets.

**Stratify**: The **"stratify" parameter is used to maintain the proportional class distribution in both the training and test sets.** It is especially useful when dealing with imbalanced datasets where the target variable has unequal class frequencies. By setting "stratify" to the target variable, the train-test split function will ensure that the resulting training and test sets have similar class distributions. This helps prevent biased model performance evaluation and ensures that each class is represented in both sets.

**Random_state**: The "random_state" parameter is **used to control the random sampling of data during the train-test split. By setting a specific value for "random_state", you can obtain the same train-test split every time you run your code. This is useful for reproducibility, as it allows you to obtain consistent results when experimenting with different models or hyperparameters**. Additionally, sharing the same "random_state" value allows others to replicate your results.



In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

**Training with Logistic Regression Model:**

Logistic Regression is a popular classification algorithm used in machine learning. Despite its name, it is primarily used for binary classification problems, where the target variable has two possible classes. However, it can be extended to handle multi-class classification as well.

The sigmoid function, also known as the logistic function, is a mathematical function that maps any real-valued number to a value between 0 and 1. It is commonly used in logistic regression and other machine learning algorithms to model the probability of an event occurring.

The sigmoid function is defined as:

σ(z) = 1 / (1 + e^(-z))

Threshold and Decision: By default, logistic regression predicts the class with the highest probability (i.e., if the probability is above 0.5, it predicts class 1; otherwise, it predicts class 0). However, you can adjust the threshold to obtain different trade-offs between precision and recall.


In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluating the Performance of the Model & its Accuracy Score on the **'Trained Data'**

In [ ]:
# Prediction On the Trained Data

X_train_prediction = model.predict(X_train)

# Accuracy Score On the Trained Data

training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [ ]:
print("Accuracy Score of the Trained Data :" ,  training_data_accuracy )

Accuracy Score of the Trained Data : 0.9865985576923076



Evaluating the Performance of the Model & its Accuracy Score on the 'Test Data'

In [ ]:
# Prediction On the Test Data

X_test_prediction = model.predict(X_test)

# Accuracy Score On the Trained Data

test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [ ]:
print("Accuracy Score of the Test Data :" ,  test_data_accuracy )

Accuracy Score of the Test Data : 0.9790865384615385


Making a **Predictive System**

In [ ]:

X_new = X_test[4]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print(" The news is Real ")
else:
  print(" The news is Fake ")

[0]
 The news is Real 


In [ ]:
print(Y_test[4])

0


Completed..!!